In [1]:
import tensorflow as tf
import logging
import json
from pathlib import Path
from datetime import datetime

from src.model.calculations.treshholds_calculations import evaluate_with_optimal_thresholds, ThresholdedPredictor
from src.model.tensorflow_utils import load_model, load_and_split_dataset, setup_logger, optimize_dataset

from src.utils.consts import MODELS_PATH, TF_RECORD_DATASET, TF_BUFFER_SIZE

# Setup logging
logger = setup_logger()

# Load model
model_name = "Simplified_DensNet_v2"
model = load_model(MODELS_PATH, model_name)
model_path = f"{MODELS_PATH}/{model_name}"

# Load datasets
DATASET_SIZE = 102697
batch_size = 32
shuffle_buffer_size = 1000
tfrecord_buffer_size = TF_BUFFER_SIZE
dataset_size = DATASET_SIZE
tfrecord_path = f"{TF_RECORD_DATASET}/chest_xray_data.tfrecord"
_train_ds, val_ds, test_ds = load_and_split_dataset(tfrecord_path, shuffle_buffer_size, tfrecord_buffer_size, dataset_size)

test_ds = optimize_dataset(test_ds, 32)
val_ds = optimize_dataset(val_ds, 32)

# Find optimal thresholds and evaluate
results = evaluate_with_optimal_thresholds(
    model=model,
    val_dataset=val_ds,
    test_dataset=test_ds,
    logger=logger,
    batch_size=32
)

# Save results
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = f"{model_path}/threshold_optimization_results_{timestamp}.json"

with open(output_path, "w") as f:
    json.dump(results, f, indent=2)

logger.info(f"Results saved to: {output_path}")

# Print summary
logger.info("\nOverall Metrics:")
for metric, value in results["overall_metrics"].items():
    logger.info(f"{metric}: {value:.4f}")

logger.info("\nPer-class Thresholds and F1 Scores:")
for class_name, metrics in results["thresholds"].items():
    logger.info(
        f"{class_name}: "
        f"threshold={metrics['threshold']:.3f}, "
        f"f1={metrics['f1_score']:.3f}, "
        f"precision={metrics['precision']:.3f}, "
        f"recall={metrics['recall']:.3f}"
    )

2025-03-01 11:08:51.282767: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2025-03-01 11:08:51.282805: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 48.00 GB
2025-03-01 11:08:51.282808: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 18.00 GB
I0000 00:00:1740823731.283013 3841705 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1740823731.283209 3841705 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-03-01 11:08:52 - INFO - Preparing datasets...
2025-03-01 11:08:52 - INFO - Finding optimal thresholds...
2025-03-01 11:08:52 - INFO - Processing class 1/15
2025-03-01 11:08:52.807674: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:376] The default 

2025-03-01 13:00:35.625014: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-03-01 13:00:35 - INFO - Results saved to: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DensNet_v2/threshold_optimization_results_20250301_130035.json
2025-03-01 13:00:35 - INFO - 
Overall Metrics:
2025-03-01 13:00:35 - INFO - accuracy: 0.5462
2025-03-01 13:00:35 - INFO - precision: 0.1354
2025-03-01 13:00:35 - INFO - recall: 0.8013
2025-03-01 13:00:35 - INFO - auc: 0.5211
2025-03-01 13:00:35 - INFO - f1: 0.3749
2025-03-01 13:00:35 - INFO - 
Per-class Thresholds and F1 Scores:
2025-03-01 13:00:35 - INFO - class_0: threshold=0.344, f1=0.204, precision=0.114, recall=0.986
2025-03-01 13:00:35 - INFO - class_1: threshold=0.253, f1=0.061, precision=0.037, recall=0.182
2025-03-01 13:00:35 - INFO - class_2: threshold=0.303, f1=0.099, precision=0.055, recall=0.522
2025-03-01 13:00:35 - INFO - class_3: threshol